# Import all library needed

In [23]:

%pip install openai pytest pandas python-dotenv scikit-learn

import openai
import pandas as pd
import pytest
import json
from dotenv import load_dotenv
import os
from sklearn.model_selection import train_test_split
import base64
import re

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 21.4 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 27.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


# Analyze the data provided

In [2]:
df = pd.read_json('cura-llm-training-data.json')

In [3]:
df

,question_id,task_id,question,rubric,answer,task_title,task_content
0,8b1d6474-a38a-4456-8ac2-74ff054b9d67,41f89fd8-0d25-4aa7-848c-6840f01023da,Which fin design worked best? Why?,"{""items"": [""I can explain my response"", ""I can...","""Fin designs that have three or four triangula...",Designing your rocket,Designing your rocket Building phase Th...
1,8add417a-4108-43e6-b033-4fa72ac4f709,35d95305-bc3a-461a-ba96-83063257f32d,Which planet had the fastest and slowest orbit...,"{""items"": [""I can explain why these planets ha...","""Mercury has the fastest orbit of 88 days. Thi...",The orbits of other planets,The orbits of other planets The orbits of o...
2,89edd167-18ed-40ba-93f1-93999a800097,c4c7058a-5f65-45a2-b097-30733070e4bd,"Based on your results (ie. shape, volume, and ...","{""items"": [""I can identify the three propertie...","""Exemplar AnswerSolids - shape did not change,...",What are objects made of?,What are objects made of? What affects whet...
3,8632c092-ea9f-4ba2-bdfa-d2f335c95247,cb21ad07-eed4-49a4-9d63-5e266a0b19b0,Determine the ratio of the load force to effor...,"{""items"": [""I can explain how we know this"", ""...","""The ratio of the load force to the effort for...",Forces in action - seesaws,Forces in action - seesaws Turning forces ...
4,8422d140-5d31-4933-ae45-52f380e1213a,fe99ba29-d69a-4231-8a66-badedae898b2,Describe the shape of the Earth’s orbit around...,"{""items"": [""I can explain the shape of the Ear...","""The Earth’s orbit around the Sun is an ellipt...",Earth's orbit,Earth's orbit What is an orbit? An orbi...
...,...,...,...,...,...,...,...
112,0e4305e5-9436-42e6-aa91-0d417d0e6b7b,62531e3f-7652-4a5c-b6ff-2e774b4f05a9,What type of material would you use if you wan...,"{""items"": [""I can explain my answer"", ""I can c...","""Translucent. Only some light passes through i...",Light absorption,Light absorption Struck by light What h...
113,e2ff5709-711c-487a-b4ef-032473532b2d,59c94ba6-4402-4d28-88d2-bbe2f2e8f9a0,"If you haven't already, watch this video of a ...","{""items"": [""I can explain how this is an illus...","""When a vehicle suddenly stops or collides, th...",What is Newton’s first law of motion?,What is Newton’s first law of motion? Force...
114,b1222c04-d4f6-4862-9d01-a0db711447e4,fb74d82a-43e6-4470-93ed-6f4d55e6df32,"Based on what each group started with, which g...","{""items"": [""I can correctly identify all of th...","""Group A - Regional/rural townGroup B - Smalle...",Why does liveability differ across places?,Why does liveability differ across places? ...
115,d1907f44-d617-4487-a946-64e7c75dac2a,a03055ca-23ed-485d-ae06-6b4b8b51f811,It's 1204 CE and Genghis Khan has just defeate...,"{""items"": [""I can suggest how to forge an alli...","""Students could take different approaches for ...",How did Genghis Khan unify the Mongols?,How did Genghis Khan unify the Mongols? Who...


In [4]:
print("question_id count: ",len(set(df['question_id'])))
print("question count: ",len(set(df['question'])))
print("answer count: ",len(set(df['answer'])))
print("task_id count: ",len(set(df['task_id'])))
print("task_title count: ",len(set(df['task_title'])))
print("task_content count: ",len(set(df['task_content'])))
print("rubric count: ",len(set(df['rubric'])))

question_id count:  117
question count:  117
answer count:  117
task_id count:  59
task_title count:  59
task_content count:  59
rubric count:  115


In [5]:
duplicate_rubrics = df[df.duplicated(subset=['rubric'], keep=False)]

print(duplicate_rubrics)


                             question_id  \
19  70ab1af4-12c9-4847-9efb-864ad4ed896e   
82  0f99ba55-9e0b-48c2-94ee-dabcbf7ed2ed   
96  06d8bcf5-3da2-473f-95b6-ca1c894ffc83   
97  8b7edaf3-3bfb-4281-90bd-c3656aa29b20   

                                 task_id  \
19  24bc38db-25f1-4e6f-b587-36b750356bb4   
82  24bc38db-25f1-4e6f-b587-36b750356bb4   
96  a6050fdf-1fc4-4b06-8d28-fe09e5e2f686   
97  a6050fdf-1fc4-4b06-8d28-fe09e5e2f686   

                                             question  \
19  What do these sources not tell you about ostra...   
82  Combining evidence from all the sources, what ...   
96  Look at the hypothetical pedigree diagram belo...   
97  Look at the hypothetical pedigree diagram belo...   

                                               rubric  \
19  {"items": ["Makes a claim about ostracism that...   
82  {"items": ["Makes a claim about ostracism that...   
96  {"items": ["I can explain my answer using the ...   
97  {"items": ["I can explain my answer usin

There is duplicated rubric,therefore check whether there is duplicate row from the data

In [6]:
df.iloc[19]['question']

'What do these sources not tell you about ostracism?'

In [7]:
df.iloc[82]['question']

'Combining evidence from all the sources, what do they tell you about ostracism?'

In [8]:
df.iloc[96]['question']

'Look at the hypothetical pedigree diagram below. It shows the inheritance of skin colour in aliens. Skin colour is inherited as an autosomal trait. Green (G) is dominant to orange (g).&nbsp;Are the affected aliens displaying the dominant or recessive trait? Give a specific reason for your answer with an example from the pedigree diagram'

In [9]:
df.iloc[97]['question']

'Look at the hypothetical pedigree diagram below. It shows the inheritance of horns in aliens. Horns are inherited as an autosomal trait. Horns (H) are dominant to no horns (h).&nbsp;Are the affected aliens displaying the dominant or recessive trait? Give a specific reason for your answer with an example from the pedigree diagram'

After check, all the data provided seems normal.

# Get the api key from .env file

In [122]:
# Load .env file
load_dotenv()

# From .env file to get the OpenAI API Key
api_key = os.getenv("OPENAI_API_KEY")

# Fine tuning

## Convert the data to fine tunning format

In [44]:
# Read the original data
with open("cura-llm-training-data.json", "r") as file:
    data = json.load(file)

# convert to chat-formatted data
with open("data/chat_fine_tuning_data.jsonl", "w") as output_file:
    for entry in data:
        # Create chat-formatted data message
        messages = [
            {"role": "system", "content": "You are an AI assistant designed to generate exemplar answers for educational purposes. Your task is to generate exemplar answers based on a provided student task context, question, and assessment rubric."},
            {"role": "user", "content": f"Task Title: {entry['task_title']}\nTask Content: {entry['task_content']}\nQuestion: {entry['question']}\nRubric: {entry['rubric']}"},
            {"role": "assistant", "content": entry['answer']}
        ]
        
        # convert to jsonl file
        jsonl_entry = {"messages": messages}
        output_file.write(json.dumps(jsonl_entry) + "\n")



## Train Test Validation Split

Using the ratio 7:1.5:1.5 to divide the original data into train, test and validation dataset for fine tunning

In [46]:
# Loading the original data and shuffle the dataset
with open("data/chat_fine_tuning_data.jsonl", "r") as file:
    data = [json.loads(line) for line in file]

# split the data into train test validation
train_data, temp_data = train_test_split(data, test_size=0.3, random_state=1025)
validation_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=1025)

# save the split data
with open("data/finetuneData/train_data.jsonl", "w") as train_file:
    for entry in train_data:
        train_file.write(json.dumps(entry) + "\n")

with open("data/finetuneData/validation_data.jsonl", "w") as validation_file:
    for entry in validation_data:
        validation_file.write(json.dumps(entry) + "\n")

with open("data/finetuneData/test_data.jsonl", "w") as test_file:
    for entry in test_data:
        test_file.write(json.dumps(entry) + "\n")

## Start Fine Tunning

In [123]:
from openai import OpenAI
client = OpenAI()

client.api_key = api_key

### Upload the file

In [47]:

# upload the train and validation data to OpenAI
response_train = client.files.create(file=open("data/finetuneData/train_data.jsonl", "rb"), purpose='fine-tune')
response_valid = client.files.create(file=open("data/finetuneData/validation_data.jsonl", "rb"), purpose='fine-tune')


In [48]:
print("Uploaded training file ID:", response_train.id)
print("Uploaded validation file ID:", response_valid.id)

Uploaded training file ID: file-yOCQ3P6liWemIXBOpy0wemMP
Uploaded validation file ID: file-CBjO3IqPHfIb6avZGg7JWQPi


### start Fine Tunning

In [49]:

# Create a fine-tuning task
response_fine_tune_v1 = client.fine_tuning.jobs.create(
    training_file=response_train.id,
    validation_file=response_valid.id,
    model="gpt-4o-mini-2024-07-18",
    hyperparameters={
    "n_epochs":2
  }
)


In [90]:
# Get the fine-tune task ID
fine_tune_job_id = response_fine_tune_v1.id
print("Fine-tuning job ID:", fine_tune_job_id)

Fine-tuning job ID: ftjob-HItchoh2yNrQxi5lPJ4o0APq


### Check the Fine tunning task

In [105]:
print(client.fine_tuning.jobs.retrieve(fine_tune_job_id).status)

succeeded


In [106]:
print(client.fine_tuning.jobs.retrieve(fine_tune_job_id))

FineTuningJob(id='ftjob-HItchoh2yNrQxi5lPJ4o0APq', created_at=1729871299, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::AMGvcji0', finished_at=1729871650, hyperparameters=Hyperparameters(n_epochs=2, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-1tLV13qTuF8etS5dptzigZgs', result_files=['file-CtR1HdBjS6gQL8CWz50nA1Vv'], seed=1296138184, status='succeeded', trained_tokens=225900, training_file='file-yOCQ3P6liWemIXBOpy0wemMP', validation_file='file-CBjO3IqPHfIb6avZGg7JWQPi', estimated_finish=None, integrations=[], user_provided_suffix=None)


In [117]:
result_file_id = client.fine_tuning.jobs.retrieve(fine_tune_job_id).result_files
print(result_file_id[0])

file-CtR1HdBjS6gQL8CWz50nA1Vv


In [83]:
client.files.content('file-CtR1HdBjS6gQL8CWz50nA1Vv')

In [94]:


# Retrieve the response from the client
response_log = client.files.content('file-CtR1HdBjS6gQL8CWz50nA1Vv')

# Get the response content (this should give you the binary content)
response_content_log = response_log.read()

# If the content is Base64 encoded, decode it:
# Assuming the content is Base64 encoded, decode it into original bytes
decoded_bytes = base64.b64decode(response_content_log)

# Write the decoded bytes to the file
with open('retrieved_file.csv', 'wb') as file:
    file.write(decoded_bytes)

# Prompt Engineering

## Get the standard answer for prompt

In [11]:
standard_question = df.iloc[1]['question']
standard_rubric = df.iloc[1]['rubric']
standard_answer = df.iloc[1]['answer']
standard_task_title = df.iloc[1]['task_title']
standard_task_content = df.iloc[1]['task_content']

In [96]:
print(standard_task_title)
print(standard_task_content)
print(standard_question)
print(standard_rubric)

print(standard_answer)

The orbits of other planets
The orbits of other planets    The orbits of other planets     All of the other planets in our solar system also orbit or revolve around the Sun.&nbsp;    Check out     NASA's live solar system         feature to have a look at the path each planet takes as it orbits around the sun. Can you find each of the planets in our solar system? Make sure   you zoom in and out to find them all!    We have already looked at Earth's orbit in detail. Each planet has its own peculiar orbit around the Sun. They are all slightly different from Earth's.       Orbits of Solar System Planets             Planet        Axial tilt        Orbit details        Seasons        Day length        Year length          Mercury       2 degrees      Egg-shaped. Between 47 and 70 million kms from the Sun      Small angle tilt means Mercury revolves the Sun almost perfectly upright. No real seasons      58 Earth days      88 Earth days         Venus       3 degrees      Nearly a perfect circ

## Estimate max token number

In [13]:


def word_count(text):
    # Using the regularization expression to remove the punctuation
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    words = cleaned_text.split()
    return len(words)

min_count = min(df['answer'].apply(word_count).tolist())
max_count = max(df['answer'].apply(word_count).tolist())




In [15]:
print("min count: ", min_count)
print("max count: ", max_count) 

min count:  1
max count:  203


In [20]:
df.iloc[59]

question_id                  2f8d9ef4-3fb1-4b81-99e3-989bbdd8ffdf
task_id                      0a0d4334-c400-4f04-b38c-0b5715232731
question        Draw a diagram of the two steps you used to se...
rubric          {"items": ["I can include all of the labels in...
answer                                                   "&nbsp;"
task_title                                 Sieving and filtration
task_content    Sieving and filtration    Sieving     Sieving ...
Name: 59, dtype: object

## Generate the prompt

In [97]:
prompt_message = f''' You are an AI assistant designed to generate exemplar answers for educational purposes. Your task is to generate exemplar answers based on a provided student task context, question, and assessment rubric. The exemplar answers should:

1. Clearly respond to the question by using information given in the student task context.
2. Align with the criteria specified in the assessment rubric to ensure quality and correctness.
3. Include relevant explanations, reasoning, and details that demonstrate a deep understanding of the topic.
4. Serve as a reliable reference point for teachers when evaluating student responses.
5. Be structured in a way that is clear, concise, and easy to follow, matching the language and comprehension level expected for the students.

Here is an example of the task, question, and rubric you will be working with:

(example Task Title: {standard_task_title}
example Task Content: {standard_task_content}
example question: {standard_question}
example Rubric: {standard_rubric})

Here is the official response to the example question:
(example Answer: {standard_answer})

Please generate an exemplar answer based on the provided information.
'''


In [103]:
def generate_answer(question,rubric,task_title,task_content):
    try:
        # context for GPT
        prompt = prompt_message
        question = f'''Task Title: {task_title}\nTask Content: {task_content}\nQuestion: {question}\nRubric: {rubric}; Please generate an exemplar answer based on the provided information.'''
        response = client.chat.completions.create(
            model='ft:gpt-4o-mini-2024-07-18:personal::AMGvcji0',
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": question}
            ]
        )
        answer = response.choices[0].message['content']
        return answer
    except Exception as e:
        print(f"Error generating answer: {e}")
        return f"There was an error generating the answer: {e}"

In [108]:
generate_answer(standard_question,standard_rubric,standard_task_title,standard_task_content)

Error generating answer: Error code: 403 - {'error': {'message': 'Project `proj_Nhn5J05hEJVxN9B3zTegcZ6x` does not have access to model `ft:gpt-4o-mini-2024-07-18:personal::AMGvcji0`', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}


"There was an error generating the answer: Error code: 403 - {'error': {'message': 'Project `proj_Nhn5J05hEJVxN9B3zTegcZ6x` does not have access to model `ft:gpt-4o-mini-2024-07-18:personal::AMGvcji0`', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}"

In [116]:
from openai import OpenAI
client = OpenAI()

client.models.list()

SyncPage[Model](data=[Model(id='gpt-4o-mini', created=1721172741, object='model', owned_by='system')], object='list')

In [125]:
client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-HItchoh2yNrQxi5lPJ4o0APq', created_at=1729871299, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::AMGvcji0', finished_at=1729871650, hyperparameters=Hyperparameters(n_epochs=2, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-1tLV13qTuF8etS5dptzigZgs', result_files=['file-CtR1HdBjS6gQL8CWz50nA1Vv'], seed=1296138184, status='succeeded', trained_tokens=225900, training_file='file-yOCQ3P6liWemIXBOpy0wemMP', validation_file='file-CBjO3IqPHfIb6avZGg7JWQPi', estimated_finish=None, integrations=[], user_provided_suffix=None), FineTuningJob(id='ftjob-MvTKG5ijt5phyliFNwopWdyR', created_at=1729870132, error=Error(code='invalid_training_file', message='The job failed due to an invalid training file. Invalid file format. Input file file-78XAD1TDfW32ZSy7VNedOHRg is in the prompt-completion format, but the speci